In [20]:
from configparser import ConfigParser
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
import pandas as pd
import numpy as np
import pickle
from sklearn.preprocessing import MinMaxScaler
from get_timbre import get_timbre
from evaluation import evaluate_data


parser = ConfigParser()
parser.read('./spotify_credentials.cfg')

SPOTIPY_CLIENT_ID = parser.get('spotify', 'SPOTIPY_CLIENT_ID')
SPOTIPY_CLIENT_SECRET = parser.get('spotify', 'SPOTIPY_CLIENT_SECRET')

sp = spotipy.Spotify(client_credentials_manager=SpotifyClientCredentials(client_id=SPOTIPY_CLIENT_ID, client_secret=SPOTIPY_CLIENT_SECRET))

user_id = 'czdoifmfngjhvoetavlok9dg5'
playlists = sp.user_playlists(user_id)
playlists = playlists['items']





playlist_id = '3YwPUeHZqBrFzLzP5BF8ND'





results = sp.playlist_tracks(playlist_id)
tracks = results['items']
track_ids = []
for track in tracks:
    track_ids.append(track['track']['id'])
track_ids

columns_to_remove = ['analysis_url', 'type', 'uri', 'track_href']
headings = ['danceability',
'energy',
'key',
'loudness',
'mode',
'speechiness',
'acousticness',
'instrumentalness',
'liveness',
'valence',
'tempo',
'id',
'duration_ms',
'time_signature',
'track_title',
'album_title',
'album_artist',
'track_number',
'total_tracks']

playlist_data = pd.DataFrame(columns=headings)

playlist_length = len(track_ids)

i = 0
for track in track_ids:
    results = sp.audio_features(track)
    if results[0]: 
        features = results[0]
    
    features_matrix = pd.DataFrame.from_records(features, index=[0])
    
    # Remove unneeded columns
    features_matrix.drop(columns = columns_to_remove, axis = 1, inplace = True)
    # print(features_matrix)

    # Add track title
    track_title = tracks[i]['track']['name']
    features_matrix['track_title'] = track_title

    # Add album title
    album_title = tracks[i]['track']['album']['name']
    features_matrix['album_title'] = album_title

    # Add artist name
    name = tracks[i]['track']['album']['artists'][0]['name']
    features_matrix['album_artist'] = name

    # Add track number and total tracks
    features_matrix['track_number'] = tracks[i]['track']['track_number']
    features_matrix['total_tracks'] = playlist_length

    # Add album as a row to the data df
    # print(features_matrix,'\n') 

    playlist_data = pd.concat([playlist_data, features_matrix])
    i += 1

playlist_data = playlist_data.reset_index(drop=True)
# playlist_data.to_csv('./data/playlist_data.csv')
playlist_data_full = pd.DataFrame(playlist_data)

timbre = pd.DataFrame(columns = ['song_timbre', 'song_timbre_start', 'song_timbre_end', 'loudness_start', 'loudness_end'])

i = 0
for track in track_ids:
    print('Getting timbre for: '+ track)
    track_timbre = get_timbre(track, playlist_data_full)
    timbre = pd.concat([timbre, track_timbre], axis=0)
    print('Timbre obtained.\n')

# playlist_data = pd.concat([playlist_data, timbre], axis=0)
timbre = timbre.reset_index(drop=True)

playlist_data = pd.concat([playlist_data, timbre], axis=1)



# Remove unnecessary testing columns
columns_to_remove = ['id',
                    'track_title',
                    'album_title',
                    'album_artist',
                    'track_number',
                    'total_tracks',
                    'key',
                    'mode',
                    'duration_ms',
                    'time_signature']
playlist_data.drop(columns = columns_to_remove, axis = 1, inplace = True)

Getting timbre for: 3js9UOPVsguIDI2WyycfRJ
Timbre obtained.

Getting timbre for: 26AuyrZGzWWiYZPSd3XBIg
Timbre obtained.

Getting timbre for: 3rnBqIIoxOpQ0p9BeW3NT4
Timbre obtained.

Getting timbre for: 22jEGuVPtvtMqKPuXdOVCh
Timbre obtained.

Getting timbre for: 77Dn6Y5SzjCzfXLjy89dYB
Timbre obtained.

Getting timbre for: 5y2K7jr9M8t1HRAu9jzssT
Timbre obtained.

Getting timbre for: 6JsfvqClYnc2R5yZ67C2ch
Timbre obtained.

Getting timbre for: 2DI0fZ4QZmLtapszYaoG6F
Timbre obtained.

Getting timbre for: 1LYipMFKz8X6fNU7RJvvVW
Timbre obtained.

Getting timbre for: 0KQh7AuuZvpTKWhcJa8Pbr
Timbre obtained.



In [22]:
original_order_eval = evaluate_data(playlist_data_full)
original_order_eval

C:\Users\crisp\anaconda3\envs\AI\lib\site-packages\pandas\core\series.py:1056: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  cacher_needs_updating = self._check_is_chained_assignment_possible()


[0.4, 0.2, 0.1, 0.0, 0.3, 0.0, 0.1, 0.0, 0.1]

In [23]:
## Load trained model
model = pickle.load(open('./regression/random_forest.sav', 'rb')) # change to whichever model we want to use
playlist_data_full['order'] = model.predict(playlist_data.values)
playlist_data_full = playlist_data_full.sort_values(by=['order'])
playlist_data_full['order'] = np.arange(1, playlist_data_full.shape[0]+1) # convert order to integer playlist track number
playlist_data_full

,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,id,duration_ms,time_signature,track_title,album_title,album_artist,track_number,total_tracks,order
5,0.859,0.625,1,-5.437,1,0.0914,0.00538,0.000124,0.1840,0.6360,149.948,5y2K7jr9M8t1HRAu9jzssT,219737,4,No Security,No Security,Skepta,1,10,1
8,0.643,0.445,9,-15.230,0,0.0430,0.68200,0.931000,0.0870,0.0714,105.000,1LYipMFKz8X6fNU7RJvvVW,165653,4,Last Kiss,Glorious,ARTY,8,10,2
6,0.632,0.805,0,-5.188,1,0.0450,0.06460,0.000058,0.1610,0.4290,93.009,6JsfvqClYnc2R5yZ67C2ch,217793,4,"Cross My Mind, Pt. 2 (feat. Kiiara)",Cross My Mind: The Mixtape,A R I Z O N A,1,10,3
1,0.620,0.627,9,-5.889,1,0.0259,0.03170,0.000000,0.1720,0.1640,90.955,26AuyrZGzWWiYZPSd3XBIg,224280,4,Bloodstream,Memories...Do Not Open,The Chainsmokers,3,10,4
4,0.753,0.782,7,-5.826,1,0.1180,0.01060,0.000004,0.1110,0.1500,100.006,77Dn6Y5SzjCzfXLjy89dYB,184800,4,Broken Child (feat. JDAM & LissA),Broken Child (feat. JDAM & LissA),Not Your Dope,1,10,5
9,0.895,0.453,0,-13.987,1,0.0756,0.00196,0.374000,0.0982,0.2570,122.151,0KQh7AuuZvpTKWhcJa8Pbr,239227,4,Funkytown,Casablanca Records Greatest Hits,Various Artists,5,10,6
2,0.554,0.864,2,-4.403,1,0.0889,0.00166,0.000000,0.1190,0.3050,126.975,3rnBqIIoxOpQ0p9BeW3NT4,201260,4,Back To Life,Back To Life,DubVision,1,10,7
3,0.671,0.874,0,-4.188,1,0.0448,0.08290,0.000027,0.2940,0.7470,109.960,22jEGuVPtvtMqKPuXdOVCh,152203,4,SAD (Clap Your Hands),SAD / Scatterbrain,Young Rising Sons,1,10,8
7,0.280,0.449,11,-7.736,1,0.0292,0.63800,0.701000,0.1050,0.1030,136.400,2DI0fZ4QZmLtapszYaoG6F,236093,4,Atlas - From “The Hunger Games: Catching Fire”...,Atlas (From “The Hunger Games: Catching Fire” ...,Coldplay,1,10,9
0,0.341,0.293,9,-12.408,1,0.0356,0.75600,0.000023,0.0833,0.1780,93.294,3js9UOPVsguIDI2WyycfRJ,249213,4,Doing Alright - Remastered 2011,Queen,Queen,2,10,10


In [44]:
from numpy import matrix

sorted_order_eval = evaluate_data(playlist_data_full)
current = matrix(sorted_order_eval)
previous = matrix(original_order_eval)
(current - previous)*100/previous # percent change

C:\Users\crisp\AppData\Local\Temp/ipykernel_2896/2315255301.py:6: RuntimeWarning: divide by zero encountered in true_divide
  (current - previous)*100/previous # percent change


matrix([[ -50.        ,   50.        ,    0.        ,           inf,
          -66.66666667,           inf, -100.        ,           inf,
            0.        ]])

0.4

In [45]:
from sklearn.metrics import mean_squared_error
y_true = playlist_data_full['track_number']
y_pred = playlist_data_full['order']
mean_squared_error(y_true, y_pred)

27.1

Rearrange

In [7]:
# get permissions to rearrange
from spotipy.oauth2 import SpotifyOAuth
scope = "playlist-modify-public"
sp = spotipy.Spotify(auth_manager=SpotifyOAuth(scope=scope,redirect_uri='http://localhost:5678/',client_id=SPOTIPY_CLIENT_ID, client_secret=SPOTIPY_CLIENT_SECRET))


# rearrange
sorted_ids = list(playlist_data_full['id'])

UPDATED_PLAYLIST = sp.playlist_replace_items(playlist_id,sorted_ids)
UPDATED_PLAYLIST

HTTP Error for PUT to https://api.spotify.com/v1/playlists/3YwPUeHZqBrFzLzP5BF8ND/tracks with Params: {} returned 403 due to User not registered in the Developer Dashboard


SpotifyException: http status: 403, code:-1 - https://api.spotify.com/v1/playlists/3YwPUeHZqBrFzLzP5BF8ND/tracks:
 User not registered in the Developer Dashboard, reason: None